In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import keras
import random
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D, CuDNNLSTM, Flatten
from keras.layers import AveragePooling1D, MaxPooling1D, Bidirectional, GlobalMaxPool1D, Concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D,concatenate
from keras.layers import SpatialDropout1D
from keras.models import Sequential
from keras.callbacks import Callback
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import os
from math import floor
import warnings
from keras import backend as K
warnings.filterwarnings('ignore')

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
random.seed(12345)


Found GPU at: /device:GPU:0


Using TensorFlow backend.


In [2]:
# If you want to train the model by yourself

# train data
with open('new_data_filter0.pickle', 'rb') as f:
    Data = pickle.load(f)
    Data = pd.DataFrame(Data)

numtrain = 186 * 6 * 12
numval = 2678

# Train validation split
idx = random.sample(range(numtrain), numval) # 80% for training
all_index = list(np.setdiff1d(range(numtrain), idx))

# Getting X_train, X_val, y_train and scale them

X_train = Data.iloc[all_index,:].drop(2251, axis = 1)
X_val = Data.iloc[idx,:].drop(2251, axis = 1)
y_train = pd.get_dummies(Data.iloc[all_index,:][2251])

names = y_train.columns
mapping = {}
i = 0
for n in names:
    mapping[i] = n
    i+=1

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [48]:
X_train.shape

(10714, 2251)

In [73]:
# Construct CNN layers
# more para-LeNet-5
cnn_model = Sequential()
cnn_model.add(Conv1D(6, 5, strides = 1, activation = 'relu', input_shape = (2251, 1)))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(16, 5, strides = 1, activation = 'relu'))
cnn_model.add(Conv1D(16, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(32, 5, strides = 1, activation = 'relu'))
cnn_model.add(Conv1D(32, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(64, 5, strides = 1, activation = 'relu'))
cnn_model.add(Conv1D(64, 5, strides = 1, activation = 'relu'))
# cnn_model.add(Conv1D(64, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

# cnn_model.add(Conv1D(128, 5, strides = 1, activation = 'relu'))
# cnn_model.add(Conv1D(128, 5, strides = 1, activation = 'relu'))
# cnn_model.add(MaxPooling1D(2, strides = 2))
# cnn_model.add(Dropout(0.2))

cnn_model.add(Flatten())
cnn_model.add(Dense(120, activation = 'relu'))
cnn_model.add(Dense(84, activation = 'relu'))
cnn_model.add(Dense(186, activation = 'softmax', activity_regularizer = keras.regularizers.l2(0.1)))
cnn_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [74]:
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_92 (Conv1D)           (None, 2247, 6)           36        
_________________________________________________________________
max_pooling1d_53 (MaxPooling (None, 1123, 6)           0         
_________________________________________________________________
dropout_53 (Dropout)         (None, 1123, 6)           0         
_________________________________________________________________
conv1d_93 (Conv1D)           (None, 1119, 16)          496       
_________________________________________________________________
conv1d_94 (Conv1D)           (None, 1115, 16)          1296      
_________________________________________________________________
max_pooling1d_54 (MaxPooling (None, 557, 16)           0         
_________________________________________________________________
dropout_54 (Dropout)         (None, 557, 16)           0         
__________

In [75]:
# Fitting CNN model
numrest = numtrain-numval
cnn_model.fit(X_train.reshape(numrest, 2251, 1), y_train.values, batch_size = 256, epochs = 54, verbose = 1)

# Validating the CNN modela
pred = cnn_model.predict(X_val.reshape(numval, 2251, 1))
out = [np.argmax(p) for p in pred]
out = np.array(out)

# Create category mapping dictionary
y_val = Data.iloc[idx,:][2251]
    
# Calculating accuracy
predicted = np.vectorize(mapping.get)(out)
print('Accuracy: '+ str(round(np.sum(predicted == y_val)/numval,2) * 100) + '%')

Epoch 1/54
10714/10714 [==============================] - 3s 255us/step - loss: 4.2654 - acc: 0.3500
Epoch 2/54
10714/10714 [==============================] - 1s 121us/step - loss: 2.9392 - acc: 0.8593
Epoch 3/54
10714/10714 [==============================] - 1s 120us/step - loss: 2.8012 - acc: 0.9046
Epoch 4/54
10714/10714 [==============================] - 1s 121us/step - loss: 2.7550 - acc: 0.9227 0s - loss: 2.7613 - acc
Epoch 5/54
10714/10714 [==============================] - 1s 121us/step - loss: 2.7285 - acc: 0.9285 0s - loss: 2.7344 - acc: 0. - ETA: 0s - loss: 2.7323 - acc: 0.
Epoch 6/54
10714/10714 [==============================] - 1s 121us/step - loss: 2.7116 - acc: 0.9354
Epoch 7/54
10714/10714 [==============================] - 1s 121us/step - loss: 2.7092 - acc: 0.9377
Epoch 8/54
10714/10714 [==============================] - 1s 121us/step - loss: 2.6915 - acc: 0.9392 0s - loss: 2.6963 - acc
Epoch 9/54
10714/10714 [==============================] - 1s 121us/step - loss: 2

In [78]:
#Or you can load our best model

from keras.models import load_model 
#cnn_model= load_model('best_all-top3-except-two-zif8-top4.h5')

cnn_model= load_model('shallow_VGG_16.h5')

# cnn_model= load_model('lenet_1319_1519.h5')


In [67]:
# cnn_model.save("lenet_1119_1719_top3_1919.h5")

In [79]:
Test = pd.read_csv('test_data_19.csv')

X_test = scaler.transform(Test.iloc[:,1:])
pred = cnn_model.predict(X_test.reshape(19, 2251, 1))
out = [np.argmax(p) for p in pred]
out = np.array(out)
predicted = np.vectorize(mapping.get)(out)

preds = []
for p in pred:
  P = []
  for i in np.argsort(p)[::-1][0:10]:
    P.append(mapping[i])
  preds.append(P)

for p in preds:
  print(p)

['NUMMOX', 'DUT-12', 'LAMGUB', 'KEXFIC10', 'QEFNAQ', 'HUYKUH', 'WABWAY', 'AFEHUO', 'JARMEU10', 'ZUYWAR']
['IDIWOH04', 'JARMEU10', 'JARMEU', 'DUT-12', 'IDIWOH06', 'ZIF-67', 'HUYKIV', 'NUMMOX', 'ZUYWAR', 'ZIF-90']
['NUMMOX', 'KEXFIC10', 'QEFNAQ', 'Eu', 'ZIF-8', 'ZIF-11', 'CIDNIM', 'WABWAY', 'WABVIF', 'BAHGUN']
['MINVUA01', 'ZIF-7', 'JARMEU10', 'ZIF-9', 'DUT-12', 'FALLOT', 'JOSNAG', 'ZIF-73', 'TILWEQ', 'ZIF-71']
['DUT-10', 'MINVUA01', 'ZIF-7', 'JARMEU10', 'ZIF-72', 'BIMDIL', 'JOKXAI', 'DUT-12', 'ZIF-5', 'F-MOF-11']
['DUT-10', 'JARMEU10', 'MINVUA01', 'ZIF-7', 'SUDBOI', 'JOKXAI', 'TILWEQ', 'F-MOF-8', 'DEFKUU', 'OGOJOJ']
['ZIF-90', 'IDIWIB', 'PIZJEN', 'IDIWOH04', 'BIMDIL', 'ITUVEZ', 'ZIF-12', 'EDIKEH', 'ZIF-8', 'PUTYEI']
['ZIF-90', 'ZIF-65', 'IDIWOH04', 'AFIXES', 'AFIXAO', 'CIDNIM', 'WIWQUO', 'PIZJEN', 'BIMDIL', 'MOF-5']
['IDIWIB', 'EDIKEH', 'AFIXES', 'IDIWOH04', 'ZIF-74', 'PUTYEI', 'MIHBAG', 'WIWQUO', 'QOPHEI', 'HODWUS']
['ZIF-90', 'ZIF-65', 'AFIXES', 'IDIWOH04', 'AFIXAO', 'CIDNIM', 'MOF-5'

In [77]:
Test = np.load('4_zif9_200.npy')
Test = pd.DataFrame(Test)
Test = Test.iloc[:,:2251]
X_test = scaler.transform(Test)
pred = cnn_model.predict(X_test.reshape(4, 2251, 1))
out = [np.argmax(p) for p in pred]
out = np.array(out)
predicted = np.vectorize(mapping.get)(out)

preds = []
for p in pred:
  P = []
  for i in np.argsort(p)[::-1][0:10]:
    P.append(mapping[i])
  preds.append(P)

for p in preds:
  print(p)

['ZIF-74', 'QEFNAQ', 'TENQOS', 'QAYTOZ', 'QOPHEI', 'XAYMUF', 'XOTHAP', 'QIJLAW', 'WABVOL', 'ENCUCC10']
['AFEHUO', 'QEFNAQ', 'ZIF-9', 'IDIWOH05', 'RASGAT', 'QAYTOZ', 'IDIWOH06', 'ZIF-73', 'KUSZED', 'TILVOZ']
['QIJLAW', 'ZIF-9', 'IDIWOH03', 'ZIF-74', 'QAYTOZ', 'QEFNAQ', 'RASGAT', 'IDIWOH07', 'KUSZED', 'ZIF-7']
['QEFNAQ', 'IDIWOH07', 'AFEHUO', 'ZIF-9', 'ZIF-7', 'KUSZED', 'IDIWOH01', 'RASGAT', 'ZIF-73', 'QAYTOZ']


In [81]:
#cnn_model.save(".h5")

In [71]:
print(pd.read_csv('label_19.csv'))


     0  ZIF-67
0    1  ZIF-67
1    2  ZIF-67
2    5  ZIF-71
3    6  ZIF-71
4    7  ZIF-71
5    8   ZIF-8
6    9   ZIF-8
7   10   ZIF-8
8   11   ZIF-8
9   16  ZIF-90
10  17  ZIF-90
11  18  ZIF-90
12   3   ZIF-7
13   4   ZIF-7
14  12   ZIF-9
15  13   ZIF-9
16  14   ZIF-9
17  15   ZIF-9


In [68]:
Test = pd.read_csv('test_data_14afterFFT.csv')

X_test = scaler.transform(Test.iloc[:,1:])
pred = cnn_model.predict(X_test.reshape(13, 2251, 1))
out = [np.argmax(p) for p in pred]
out = np.array(out)
predicted = np.vectorize(mapping.get)(out)

preds = []
for p in pred:
  P = []
  for i in np.argsort(p)[::-1][0:10]:
    P.append(mapping[i])
  preds.append(P)

for p in preds:
  print(p)

['JARMEU10', 'JARMEU', 'ZIF-67', 'QAMXIL', 'DUT-12', 'ZIF-71', 'JOKXEM', 'UHAXUW', 'MINVUA', 'LAMFEK']
['ZIF-67', 'JARMEU10', 'JARMEU', 'DUT-12', 'DUT-10', 'QAMXIL', 'LAMFEK', 'QALWUV', 'ZIF-71', 'NUMMOX']
['JARMEU10', 'ZIF-67', 'JARMEU', 'HUYKIV', 'UHAXUW', 'DUT-12', 'ZIF-71', 'REGJIW02', 'WABVOL', 'JOKXEM']
['DUT-10', 'ZIF-71', 'ZIF-9', 'ZIF-75', 'ZIF-12', 'AFEHUO', 'MIL-101', 'ZIF-72', 'JOKYEN', 'YARYEV']
['DUT-10', 'ZIF-71', 'ZIF-9', 'ZIF-75', 'ZIF-12', 'AFEHUO', 'ZIF-72', 'JOKYEN', 'REGJIW02', 'MIL-101']
['ZIF-71', 'DUT-10', 'ZIF-9', 'ZIF-75', 'ZIF-72', 'AFEHUO', 'MIL-101', 'ZIF-12', 'JOKYEN', 'QEFNAQ']
['ZIF-90', 'ZIF-8', 'ZIF-67', 'JOKXEM', 'AFIXES', 'PIZJEN', 'IDIWOH', 'VOCQAF', 'JOKYAJ10', 'JOKXEM10']
['ZIF-90', 'ZIF-8', 'ZIF-67', 'PIZJEN', 'JOKXEM', 'AFIXES', 'JOKYAJ10', 'IDIWIB', 'VOCQAF', 'RASGAT']
['ZIF-67', 'ZIF-8', 'ZIF-90', 'JOKXEM', 'AFIXES', 'MIMVEJ', 'QOJTUE', 'PIZJEN', 'ZIF-81', 'PUTYEI']
['ZIF-90', 'ZIF-8', 'ZIF-67', 'JOKXEM', 'MOF-5', 'IDIWOH', 'VOCQAF', 'ZIF-65',

In [36]:
max(X_test[0])

1.6431448714773906

In [26]:
max(X_test[0])

2.54536888654884

In [ ]:
with open('T_origin19.pickle', 'rb') as f:
    Data19 = pickle.load(f)
    Data19 = pd.DataFrame(Data19)


In [20]:
with open('T_origin19.pickle', 'rb') as f:
    Data19 = pickle.load(f)
    Data19 = pd.DataFrame(Data19)

Test = pd.DataFrame(Data19)
Test = Test.iloc[:,:2251]
X_test = scaler.transform(Test)
pred = cnn_model.predict(X_test.reshape(19, 2251, 1))
out = [np.argmax(p) for p in pred]
out = np.array(out)
predicted = np.vectorize(mapping.get)(out)

preds = []
for p in pred:
  P = []
  for i in np.argsort(p)[::-1][0:5]:
    P.append(mapping[i])
  preds.append(P)

for p in preds:
  print(p)

['MOF-5', 'ZIF-7', 'ZUYWAR', 'JOKXUC', 'JOKYOX']
['MOF-5', 'AFIXES', 'JOKXUC', 'JOKYOX10', 'JOKYOX']
['MOF-5', 'JOKXUC', 'JOKYOX10', 'JOKYOX', 'JOKYIR10']
['ZIF-7', 'ZUYWAR', 'JOSNAG', 'JOKYOX', 'JOKYIR10']
['ZIF-7', 'ZUYWAR', 'JOSNAG', 'JOKYOX', 'JOKYIR10']
['ZIF-12', 'ZUYWAR', 'JOKXOW10', 'JOKYIR10', 'JOKYIR']
['ZIF-12', 'ZUYWAR', 'JOKXOW10', 'JOKYIR10', 'JOKYIR']
['ZIF-12', 'ZUYWAR', 'JOKXOW10', 'JOKYIR10', 'JOKYIR']
['ZIF-90', 'ZUYWAR', 'JOKXOW10', 'JOKYOX', 'JOKYIR10']
['ZIF-90', 'ZUYWAR', 'JOKXOW10', 'JOKYOX', 'JOKYIR10']
['ZIF-90', 'ZUYWAR', 'JOKXOW10', 'JOKYOX', 'JOKYIR10']
['ZIF-90', 'ZUYWAR', 'JOKXOW10', 'JOKYOX', 'JOKYIR10']
['ZIF-7', 'ZUYWAR', 'JOSNAG', 'JOKYOX', 'JOKYIR10']
['ZIF-7', 'ZUYWAR', 'JOSNAG', 'JOKYOX', 'JOKYIR10']
['ZIF-6', 'ZUYWAR', 'JOKXOW10', 'JOKYOX', 'JOKYIR10']
['ZIF-7', 'ZUYWAR', 'JOSNAG', 'JOKYOX', 'JOKYIR10']
['ZIF-90', 'ZUYWAR', 'JOKXOW10', 'JOKYOX', 'JOKYIR10']
['ZIF-90', 'ZUYWAR', 'JOKXOW10', 'JOKYOX', 'JOKYIR10']
['ZIF-90', 'ZUYWAR', 'JOKXOW10', 'JO

In [18]:
print(max(X_test[0]))
print(max(X_test[1]))

14119.101239181944
18068.654744826214


In [24]:
#cnn_model.save("low_noise_zif9inTop5.h5")